#Week 2 Exercises

In this weeks exercises you will use Numpy/Scipy to impliment some numerical algorithms and then you will use Pandas to perform a rudamentary data analysis using the KDD 98 dataset.  Along the way you will use unix/basic python from the first week as well as git to save your work.

As a first step we import the libraries we'll use later on.  This allows us to use numpy library calls by prefixing the call with np.

In [2]:
#Import the libraries 
import numpy as np
import scipy as sp
import pandas as pd

##Matrix Manipulations
Lets first create a matrix and perform some manipulations of it.

Using numpy's matrix data structure, define the following matricies:

$$A=\left[ \begin{array}{ccc} 3 & 5 & 9 \\ 3 & 3 & 4 \\ 5 & 9 & 17 \end{array} \right]$$

$$B=\left[ \begin{array}{c} 2 \\ 1 \\ 4 \end{array} \right]$$

After this solve the matrix equation:
$$Ax = B$$

Now write three functions for matrix multiply $C=AB$ in each of the following styles:

1. By using nested for loops to impliment the naive algorithm ($C_{ij}=\sum_{k=0}^{m-1}A_{ik}B_{kj}$)
2. Using numpy's built in martrix multiplication  
3. Using Cython

The three methods should have the same answer

In [3]:
A=np.matrix('3 5 9; 3 3 4; 5 9 17', dtype=float)
B=np.matrix('2; 1; 4', dtype=float)
#print A.shape, A.dtype, A[0,:]
#print B.shape, B.dtype, B
x = np.linalg.solve(A, B)
print 'x =', x
print

def mm1(A, B):
    if A.shape[1] != B.shape[0]:
        print "Number of columns of A must equal number of rows of B"
        return None
    C=np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
    return C

def mm2(A, B):
#    return(A * B)
    return(np.dot(A, B))

print mm1(A, B)
print
print mm2(A, B)

x = [[ 1.]
 [-2.]
 [ 1.]]

[[ 47.]
 [ 25.]
 [ 87.]]

[[ 47.]
 [ 25.]
 [ 87.]]


In [ ]:
%load_ext cythonmagic

In [ ]:
%%cython
cimport numpy as np
import numpy as np
def mm3(np.ndarray[np.float64_t, ndim=2] A, np.ndarray[np.float64_t, ndim=2] B):
    cdef np.ndarray[np.float64_t, ndim=2] C
    cdef long i, j, k
#     if A.shape[1] != B.shape[0]:
#         print "Number of columns of A must equal number of rows of B"
#         return None
    C=np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
    return C


In [ ]:
print mm3(A, B)

Now we wish to evaluate the performance of these three methods.  Write a method that given three dmiensions (a,b,c) makes a random a x b and b x c matrix and computes the product using your three functions and reports the speed of each method.

After this measure performance of each method for all $a,b,c \in \{10,100,1000,10000\}$ and plot the results.  Is one method always the fastest?  Discuss why this is or is not the case.

In [ ]:
%%time

np.random.seed(20141102)

def tmm(a, b, c):
    A=np.random.uniform(low=-1, high=1, size=(a, b))
    B=np.random.uniform(low=-1, high=1, size=(b, c))
    t1=time.time()
    mm1(A, B)
    t2=time.time()
    mm2(A, B)
    t3=time.time()
    mm3(A, B)
    t4=time.time()
    return np.array([t2-t1, t3-t2, t4-t3])

nsim=100
msize=[10, 50, 100]
avg=np.zeros(shape=(nsim, 3))
res=np.zeros(shape=(len(msize)**3, 10))
irow=0
labels=[]
for a in msize:
    for b in msize:
        for c in msize:
            labels.append(str(a)+':'+str(b)+':'+str(c))
            for k in range(nsim):
                avg[k,]=tmm(a, b, c)
            res[irow,]=np.concatenate(([irow, a, b, c], avg.mean(axis=0), avg.std(axis=0)))
            irow+=1

from textwrap import wrap

pl.figure(figsize=(8, 5))
pl.semilogy(res[:, 4], c='blue', marker='o', label='Python')
pl.semilogy(res[:, 6], c='green', marker='o', label='Cython')
pl.semilogy(res[:, 5], c='red', marker='o', label='Numpy')
pl.title('Mean of %d trials' % nsim)
pl.legend(loc="upper left", bbox_to_anchor=(1,1))
pl.xticks(range(len(labels)), labels, rotation=90)
# fnote=', '.join(labels)
# pl.xlabel('\n'.join(wrap('Dimensions: '+fnote, 60)))

# ax=gca()
# ax.set_yscale('log')
# pl.errorbar(res[:, 0], res[:, 4], res[:, 7], c='blue', label='Python')
# pl.errorbar(res[:, 0], res[:, 6], res[:, 9], c='green', label='Cython')
# pl.errorbar(res[:, 0], res[:, 5], res[:, 8], c='red', label='Numpy')
# pl.title('Mean (SD) of %d trials' % nsim)
# pl.legend(loc="upper left", bbox_to_anchor=(1,1), scatterpoints=1)
# fnote=', '.join(labels)
# pl.xlabel('\n'.join(wrap('Dimensions: '+fnote, 60)))

**BONUS** Now repeat the past two problems but instead of computing the matrix product, compute a matrix's [determinant](http://en.wikipedia.org/wiki/Determinant).  Measure performance for matricies of various sizes and discuss the results.  Determinant may get impractical to calculate for not too huge of matricies, so no need to goto 1000x1000 matricies.

In [ ]:
np.linalg.det(A)

###IO Exercises

Below is a map of various datatypes in python that you have come across and their corresponding JSON equivalents.

$$Datatypes=\left[ \begin{array}{cc} JSON & Python3 \\ object & dictionary \\ array & list \\ string & string \\ integer	& integer \\ real number & float \\ true & True \\ false & False \\ null & None  \end{array} \right]$$


There are atleast two very important python datatypes missing in the above list. 
Can you find the same?  [list the two mising python datatypes in this markdown cell below]

1. tuple
2. set

Now We can save the above map as a dictionary with Key-value pairs 
1. create a python dictionary named dataypes, having the above map as the Key-value pairs with Python datatypes as values and JSON equivalents as keys.
2. Save it as a pickle called datatypes and gzip the same.
3. Reload this pickle, and read the file contents and output the data in the following formatted way as given in this example - "The JSON equivalent for the Python datatype Dictionary is Object". Output similarly for the rest of the key-value pairs.
4. Save this data as a JSON but using Python datatypes as keys and JSON equivalent as values this time. 

In [5]:
import pickle
import json
import gzip
datatypes={'object':'dictionary', 'array':'list', 'string':'string', 'integer':'integer',
           'real number':'float', 'true':'True', 'false':'False', 'null':'None'}

pickle.dump(datatypes, gzip.open('datatypeszip.pkl', 'wb'))
dt=pickle.load(gzip.open('datatypeszip.pkl', 'rb'))
dtjsn={}
for key, value in dt.items():
    print 'The JSON equivalent for the Python datatype %s is %s' % (value, key)
    dtjsn[value]=key

dtjsn
json.dump(dtjsn, gzip.open('datatypeszip.jsn', 'wb'))

The JSON equivalent for the Python datatype False is false
The JSON equivalent for the Python datatype string is string
The JSON equivalent for the Python datatype float is real number
The JSON equivalent for the Python datatype dictionary is object
The JSON equivalent for the Python datatype integer is integer
The JSON equivalent for the Python datatype list is array
The JSON equivalent for the Python datatype None is null
The JSON equivalent for the Python datatype True is true


##Pandas Data Analysis
Pandas gives us a nice set of tools to work with columnar data (similar to R's dataframe). 
To learn how to use this it makes the most sense to use a real data set.
For this assignment we'll use the KDD Cup 1998 dataset, which can be sourced from http://kdd.ics.uci.edu/databases/kddcup98/kddcup98.html .


###Acquiring Data
First we pull the README file from the dataset into this notebook via the unix "curl" command.  Remember you can hide/minimize output cells via the button on the left of the output.

In [1]:
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/readme

As you can see this README describes several files which may be of use.  In particular there are two more documentation files (DOC and DIC) we should read to get an idea of the data format.  Bring these files into the notebook.

In [2]:
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98doc.txt
!curl http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98dic.txt

EPSILON CONFIDENTIAL      EPSILON CONFIDENTIAL    EPSILON CONFIDENTIAL

    INFORMATION LISTED BELOW IS AVAILABLE UNDER THE TERMS OF THE  
                      CONFIDENTIALITY AGREEMENT                

EPSILON CONFIDENTIAL      EPSILON CONFIDENTIAL    EPSILON CONFIDENTIAL

+--------------------------------------------------------------------+
|                   DOCUMENTATION TO ACCOMPANY                       |
|                                                                    |
|                          KDD-CUP-98                                |
|                                                                    |
|          The Second International Knowledge Discovery and          |
|                 Data Mining Tools Competition                      |
|                                                                    |
|                Held in Conjunction with KDD-98                     |
|                                                                    |
|          The

Now we wish to download the cup98lrn.zip file and unzip it into a new subdirectory called "data".  
However, since this file is pretty big we don't want to store it on github.  
Luckily git provides the [.gitignore](http://git-scm.com/docs/gitignore) file which allows us to specify files we don't want to put into our git repository.

Please do the following steps:

1. Add the directory "data" to the .gitignore file
2. Commit the new .gitignore file
3. Create a new directory "data"
4. Download http://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98lrn.zip into the data directory
5. Unzip the cup98lrn.zip (we will only be using the unzipped version, so feel free to remove the zip file)
6. Run "git status" to show that the data directory is not an untracked file (this indicates it is ignored)

**NOTE:** These steps only need to be run once, it is advised you comment all the lines out by putting a # at the start of each line after they have run.  This will save you time in the future when you have to rerun all cells/don't want to spend a few minutes downloading the data file.

Now perform some basic sanity checks on the data.  Using a combination of unix/basic python answer the following questions:

1. How many lines are there?  
2. Is the file character seperated or fixed width format?
3. Is there a header?  If so how many fields are in it?
4. Do all rows have the same number of fields as the header?
5. Does anyhting in 1-4 disagree with the readme file or indicate erroneous data?

In [ ]:
#1 determine the line count
line_count=!wc -l data/cup98LRN.txt | cut -d ' ' -f 1
print line_count[0]


# check if file is fixed width
infile = open('data/cup98LRN.txt', 'r')
first_line_length = None
fixed_width_file = True
for line in infile.readlines():
    if not first_line_length:
        first_line_length = len(line)
    elif first_line_length != len(line):
        print "Not a fixed width format file\n"
        fixed_width_file = False
        break
infile.close()
if fixed_width_file:
    print "file is in a fixed width format\n"


#3 check if a header exists and how many fields are in it
!head -n 2 data/cup98LRN.txt

#4 check if all of the rows have the same number of fields
infile = open('data/cup98LRN.txt', 'r')
count = None
malformed_line_count = 0
for line in infile.readlines():
    current_line_count = len(line.split(','))
    if not count:
        count = current_line_count
    elif current_line_count != count:
        malformed_line_count += 1
if malformed_line_count == 0:
    print "\n#4 yes, all lines are equal"
else:
    print "\n#4 no, all lines ARE NOT equal"
infile.close()

# remove null bytes from the file
!sed -i 's/[\x0]//g' data/cup98LRN.txt

Give answers to questions 1-4 in this markdown cell:

1. 95413 lines
2. character separated using a comma
3. Yes there is a header
4. Yes, all lines are equal


Now load the data file into a pandas data frame called "learn".  To save some time, we've loaded the data dictionary into col_types.  

Finally split learn into two data frames, learn_y: the targets (two columns described in the documentation) and learn_x: the predictors (everything but the targets)

In [ ]:
dict_file = open("dict.dat")
col_types = [ (x.split("\t")[0], x.strip().split("\t")[1]) for x in dict_file.readlines() ]

In [ ]:
learn = pd.read_csv('data/cup98LRN.txt', low_memory=False)
learn_x = learn[['TARGET_B', 'TARGET_D']]
learn_y = learn.drop(['TARGET_B', 'TARGET_D'], axis=1)

###Summarizing Data
Now that we have loaded data into the learn table, we wish to to summarize the data.  
Write a function called summary which takes a pandas data frame and prints a summary of each column containing the following:

If the column is numeric:

1. Mean
2. Standard Deviation
3. Min/Max
4. Number of missing values (NaN, Inf, NA)

If the column is non alphabetical:

1. Number of distinct values
2. Number of missing values (NaN, INF, NA, blank/all spaces)
3. The frequency of the 3 most common values and 3 least common values

Format the output to be human readable.

For example:
> Field_1  
> mean: 50  
> std_dev: 25  
> min: 0  
> max: 100  
> missing: 5
>  
> Field_2  
> distinct_values: 100  
> missing: 10  
>  
> 3 most common:  
>   the: 1000  
>   cat: 950  
>   meows: 900  
>  
> 3 least common:  
>   dogs: 5  
>   lizards: 4  
>   eggs: 1  

In [ ]:
 def summary(df):
    numeric = df._get_numeric_data()
    mean_result = df.mean(numeric_only=True)
    std_dev_result = df.std(numeric_only=True)
    min_result = df.min(numeric_only=True)
    max_result = df.max(numeric_only=True)
    
    for field in df:
        print field
        if field in numeric:
            print "mean: %d" % mean_result[field]
            print "std_dev: %d" % std_dev_result[field]
            print "min: %d" % min_result[field]
            print "max: %d" % max_result[field]
            print "missing: %d" % (np.isinf(df[field]) | df[field].isnull()).sum()
        else:
            print "distinct_values: %d" % df[field].nunique()
            print "missing: %d\n" % (df[field].apply(lambda x: x == np.inf or x == -np.inf or str(x).strip() == '') | df[field].isnull()).sum()
            print "3 most common:"
            cleaned_data = df.dropna(subset=[field])
            value_counts = cleaned_data[field].value_counts()
            print value_counts.head(3)
            print "\n 3 least common:"
            print value_counts.tail(3)
        print ""

summary(learn)

 ### Pandas analysis on Calit2 data 

Import data from http://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/CalIt2.data using curl

This data comes from the main door of the CalIt2 building at UCI. Observations come from 2 data streams (people flow in and out of the building), over 15 weeks, 48 time slices per day (half hour count aggregates).

Attribute Information:
1. Flow ID: 7 is out flow, 9 is in flow
2. Date: MM/DD/YY
3. Time: HH:MM:SS
4. Count: Number of counts reported for the previous half hour


In [3]:
!curl http://archive.ics.uci.edu/ml/machine-learning-databases/event-detection/CalIt2.data > CalIt2.data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218k  100  218k    0     0  1523k      0 --:--:-- --:--:-- --:--:-- 1525k


#### Selecting Data ####
1. Select all data for the date July 24 2005 having flow id=7. Also output the row count of results 
2. Select all rows whose count is greater than 5. Sort the result on count in descending order and output the top 10 rows

In [ ]:
cal_id_data = pd.read_csv('CalIt2.data', low_memory=False, header=None)
cal_id_data.columns = ['Flow_ID', 'Date', 'Time', 'Count']
rows = cal_id_data[(cal_id_data['Date'] == '07/24/05') & (cal_id_data['Flow_ID'] == 7)]
print len(number1['Count'])
print rows
rows2 = cal_id_data[cal_id_data['Count'] > 5].sort('Count', ascending=False)[0:10]
print rows2

#### Apply function ####
1. For the 10 rows outputted above, use Pandas Apply function to subtract lowest value of the 10 from all of them and then output the average value of the resulting counts
2. On the entire data, use apply function to sum all counts with flow_id=9 and date is 07/24/05

In [ ]:
rows2.Count.apply(lambda x: x - rows2.Count.min()).mean()
import datetime

def select(row):
    if row['Date'] == '07/24/05' and row['Flow_ID'] == 9:
        return True
    else:
        return False

print cal_id_data[cal_id_data.apply(lambda x: select(x), axis=1)].apply(np.sum)['Count']

#### Grouping ####
1. Select data in the month of August 2005 having flow id=7
2. Group the data based on date and get the max count per date

In [ ]:
g1 = cal_id_data[(cal_id_data['Flow_ID'] == 7) & (cal_id_data.Date.str.startswith('08')) & (cal_id_data.Date.str.endswith('05'))]
print g1

#2
g2 = g1.groupby('Date')
print g2.max()

#### Stacking, Unstacking ####
1. Stack the data with count and flow_id as indexes
2. Use reset_index to reset the stacked hierarchy by 1 level. The index then will just be the counts

In [ ]:
stack1 = cal_id_data.set_index(['Count', 'Flow_ID']).stack()
print stack1
stack2 = stack1.reset_index(level=1)
print stack2.head()